# Exploration des Données pour l'ETF Personnalisé

Ce notebook réalise une exploration des données financières pour la construction d'un ETF personnalisé ciblant les marchés émergents et les technologies.

## Objectifs
- Explorer les données de marché des actifs potentiels pour l'ETF
- Analyser les caractéristiques fondamentales des actifs
- Examiner les métriques de liquidité
- Visualiser les corrélations entre actifs

In [ ]:
# Imports nécessaires
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from pathlib import Path
import yaml
import json
from datetime import datetime, timedelta

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("viridis")

# Définir les chemins de données
data_dir = Path("../data")
raw_dir = data_dir / "raw"
processed_dir = data_dir / "processed"
config_path = Path("../config/config.yaml")

# Créer les répertoires s'ils n'existent pas
raw_dir.mkdir(parents=True, exist_ok=True)
processed_dir.mkdir(parents=True, exist_ok=True)

## Chargement de la Configuration

Commençons par charger la configuration du projet depuis le fichier YAML.

In [ ]:
# Charger la configuration
try:
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
        print("Configuration chargée avec succès!")
except Exception as e:
    print(f"Erreur lors du chargement de la configuration: {e}")
    # Créer une configuration par défaut si le fichier n'existe pas
    config = {
        "etf": {
            "name": "Innovation Tech Émergente ETF",
            "benchmark": "MSCI Emerging Markets Technology Index"
        },
        "asset_selection": {
            "markets": ["Brazil", "Russia", "India", "China", "South Africa"],
            "sectors": ["Technology", "Telecommunications", "Consumer Electronics"]
        },
        "backtesting": {
            "start_date": "2018-01-01",
            "end_date": "2022-12-31"
        }
    }

# Afficher les informations de base sur l'ETF
print(f"Nom de l'ETF: {config['etf']['name']}")
print(f"Indice de référence: {config['etf']['benchmark']}")
print(f"\nMarchés ciblés: {', '.join(config['asset_selection']['markets'])}")
print(f"Secteurs ciblés: {', '.join(config['asset_selection']['sectors'])}")
print(f"\nPériode de backtest: {config['backtesting']['start_date']} à {config['backtesting']['end_date']}")

## Collecte des Données de Marché

Pour notre ETF, nous avons besoin de données de marché pour les actifs des pays émergents ciblés. Commençons par identifier les ETFs les plus connus pour ces marchés afin d'extraire leurs composants.

In [ ]:
# Définir les ETFs représentatifs pour chaque marché
market_etfs = {
    "Brazil": "EWZ",
    "Russia": "RSX",
    "India": "INDA",
    "China": "MCHI",
    "South Africa": "EZA",
    "Mexico": "EWW",
    "Indonesia": "EIDO",
    "Turkey": "TUR"
}

# Récupérer des informations sur ces ETFs
for market, etf_ticker in market_etfs.items():
    try:
        etf = yf.Ticker(etf_ticker)
        info = etf.info
        print(f"{market} - {etf_ticker}: {info.get('longName', 'N/A')}")
        print(f"  Actifs sous gestion: {info.get('totalAssets', 'N/A'):,}")
        print(f"  Description: {info.get('longBusinessSummary', 'N/A')[:150]}...")
        print("\n")
    except Exception as e:
        print(f"Erreur lors de la récupération des informations pour {etf_ticker}: {e}")

## Analyse des Composants des ETFs Existants

Examinons la composition des ETFs pour identifier les entreprises potentielles pour notre ETF personnalisé, en se concentrant sur les secteurs technologiques.

In [ ]:
# Créer un dictionnaire pour stocker les composants par marché
market_components = {}

# Boucler sur chaque ETF de marché
for market, etf_ticker in market_etfs.items():
    try:
        print(f"Récupération des composants pour {market} ({etf_ticker})...")
        etf = yf.Ticker(etf_ticker)
        holdings = etf.get_holdings()
        
        if holdings is not None and not holdings.empty:
            # Filtrer les secteurs technologiques
            tech_keywords = ['tech', 'software', 'hardware', 'semiconductor', 'telecom', 
                           'communication', 'electronics', 'internet', 'media', 'digital']
            
            tech_holdings = holdings[holdings['Sector'].str.lower().str.contains('|'.join(tech_keywords), na=False)]
            
            print(f"  Nombre total de composants: {len(holdings)}")
            print(f"  Nombre de composants tech: {len(tech_holdings)}")
            
            # Afficher les 5 plus grandes entreprises tech
            if not tech_holdings.empty:
                print("\n  Top 5 entreprises tech par poids:")
                top5 = tech_holdings.sort_values('% of ETF', ascending=False).head(5)
                for idx, row in top5.iterrows():
                    print(f"    {row['Name']} ({row['Ticker']}) - {row['Sector']} - {row['% of ETF']:.2%}")
                
                # Stocker les composants tech
                market_components[market] = tech_holdings
            else:
                print("  Aucune entreprise tech trouvée dans cet ETF.")
        else:
            print(f"  Pas de données de holdings disponibles pour {etf_ticker}")
        
        print("\n")
    except Exception as e:
        print(f"Erreur lors de la récupération des holdings pour {etf_ticker}: {e}\n")

## Exploration des Prix Historiques

Récupérons les prix historiques pour quelques-unes des entreprises technologiques les plus importantes de chaque marché.

In [ ]:
# Créer une liste des tickers technologiques les plus importants
top_tech_tickers = []

for market, holdings in market_components.items():
    if not holdings.empty:
        # Prendre les 3 plus grands tickers tech par marché
        top3 = holdings.sort_values('% of ETF', ascending=False).head(3)
        for ticker in top3['Ticker']:
            if ticker not in top_tech_tickers and pd.notna(ticker):
                top_tech_tickers.append(ticker)

print(f"Nombre de tickers tech sélectionnés: {len(top_tech_tickers)}")
print(f"Tickers: {', '.join(top_tech_tickers)}")

# Récupérer les prix historiques
start_date = config['backtesting']['start_date']
end_date = config['backtesting']['end_date']

print(f"\nRécupération des prix historiques du {start_date} au {end_date}...")

try:
    # Télécharger les prix
    prices_df = yf.download(top_tech_tickers, start=start_date, end=end_date, auto_adjust=True)['Close']
    
    # Afficher les informations sur les données de prix
    print(f"Données de prix récupérées: {prices_df.shape[0]} jours pour {prices_df.shape[1]} tickers")
    
    # Sauvegarder les données de prix
    prices_path = raw_dir / "selected_prices.parquet"
    prices_df.to_parquet(prices_path)
    print(f"Données de prix sauvegardées dans {prices_path}")
except Exception as e:
    print(f"Erreur lors de la récupération des prix: {e}")
    prices_df = pd.DataFrame()

## Visualisation des Performances Historiques

Examinons les performances historiques des actifs sélectionnés pour avoir une idée des tendances et des volatilités.

In [ ]:
if not prices_df.empty:
    # Calculer les rendements cumulés
    returns = prices_df.pct_change()
    cumulative_returns = (1 + returns).cumprod() - 1
    
    # Visualiser les rendements cumulés
    plt.figure(figsize=(14, 8))
    cumulative_returns.plot()
    plt.title('Rendements Cumulés des Actifs Tech des Marchés Émergents', fontsize=16)
    plt.xlabel('Date')
    plt.ylabel('Rendement Cumulé (%)')
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    # Calculer les statistiques de performance
    perf_stats = pd.DataFrame({
        'Rendement Total': cumulative_returns.iloc[-1],
        'Rendement Annualisé': (1 + cumulative_returns.iloc[-1]) ** (252 / len(cumulative_returns)) - 1,
        'Volatilité Annualisée': returns.std() * np.sqrt(252),
        'Ratio de Sharpe': (returns.mean() - 0.02/252) / (returns.std()) * np.sqrt(252),
        'Drawdown Maximum': ((1 + returns).cumprod().div((1 + returns).cumprod().cummax()) - 1).min()
    })
    
    # Afficher les statistiques triées par rendement annualisé
    perf_stats = perf_stats.sort_values('Rendement Annualisé', ascending=False)
    display(perf_stats.style.format({
        'Rendement Total': '{:.2%}',
        'Rendement Annualisé': '{:.2%}',
        'Volatilité Annualisée': '{:.2%}',
        'Ratio de Sharpe': '{:.2f}',
        'Drawdown Maximum': '{:.2%}'
    }))

## Score Combiné pour la Sélection des Actifs

Créons un score combiné pour classifier les actifs en fonction de plusieurs critères importants pour notre ETF.

In [ ]:
if 'fundamentals_df' in locals() and not fundamentals_df.empty and not prices_df.empty:
    # Créer un DataFrame pour le scoring
    scoring_df = fundamentals_df[['Ticker', 'Nom', 'Pays', 'Secteur']].copy()
    
    # Ajouter les métriques de performance
    if 'perf_stats' in locals():
        for ticker in scoring_df['Ticker']:
            if ticker in perf_stats.index:
                scoring_df.loc[scoring_df['Ticker'] == ticker, 'Rendement Annualisé'] = perf_stats.loc[ticker, 'Rendement Annualisé']
                scoring_df.loc[scoring_df['Ticker'] == ticker, 'Volatilité'] = perf_stats.loc[ticker, 'Volatilité Annualisée']
                scoring_df.loc[scoring_df['Ticker'] == ticker, 'Ratio de Sharpe'] = perf_stats.loc[ticker, 'Ratio de Sharpe']
    
    # Ajouter les métriques fondamentales
    scoring_df['Marge Bénéficiaire'] = fundamentals_df['Marge Bénéficiaire']
    scoring_df['ROE'] = fundamentals_df['ROE']
    scoring_df['Croissance des Revenus'] = fundamentals_df['Croissance des Revenus']
    scoring_df['Capitalisation Boursière'] = fundamentals_df['Capitalisation Boursière']
    
    # Ajouter les métriques de liquidité
    if 'liquidity_df' in locals() and not liquidity_df.empty:
        for ticker in scoring_df['Ticker']:
            if ticker in liquidity_df.index:
                scoring_df.loc[scoring_df['Ticker'] == ticker, 'Volume Quotidien Moyen'] = liquidity_df.loc[ticker, 'Volume Quotidien Moyen']
                scoring_df.loc[scoring_df['Ticker'] == ticker, 'Stabilité du Volume'] = liquidity_df.loc[ticker, 'Stabilité du Volume (CV)']
    
    # Normaliser les métriques pour le scoring (min-max scaling)
    from sklearn.preprocessing import MinMaxScaler
    
    metrics_to_normalize = [
        'Rendement Annualisé', 'Volatilité', 'Ratio de Sharpe', 'Marge Bénéficiaire', 'ROE', 
        'Croissance des Revenus', 'Capitalisation Boursière', 'Volume Quotidien Moyen', 'Stabilité du Volume'
    ]
    
    for metric in metrics_to_normalize:
        if metric in scoring_df.columns:
            # Remplacer les valeurs manquantes par la médiane
            scoring_df[metric] = scoring_df[metric].fillna(scoring_df[metric].median())
            
            # Pour certaines métriques, une valeur plus faible est meilleure (volatilité, stabilité du volume)
            if metric in ['Volatilité', 'Stabilité du Volume']:
                scoring_df[f'{metric}_Score'] = 1 - MinMaxScaler().fit_transform(scoring_df[metric].values.reshape(-1, 1)).flatten()
            else:
                scoring_df[f'{metric}_Score'] = MinMaxScaler().fit_transform(scoring_df[metric].values.reshape(-1, 1)).flatten()
    
    # Calculer un score combiné (avec différentes pondérations selon l'importance des critères)
    scoring_df['Score_Performance'] = (scoring_df.get('Rendement Annualisé_Score', 0) * 0.4 + 
                                     scoring_df.get('Volatilité_Score', 0) * 0.3 + 
                                     scoring_df.get('Ratio de Sharpe_Score', 0) * 0.3)
    
    scoring_df['Score_Fondamental'] = (scoring_df.get('Marge Bénéficiaire_Score', 0) * 0.3 + 
                                     scoring_df.get('ROE_Score', 0) * 0.3 + 
                                     scoring_df.get('Croissance des Revenus_Score', 0) * 0.4)
    
    scoring_df['Score_Liquidité'] = (scoring_df.get('Capitalisation Boursière_Score', 0) * 0.3 + 
                                   scoring_df.get('Volume Quotidien Moyen_Score', 0) * 0.5 + 
                                   scoring_df.get('Stabilité du Volume_Score', 0) * 0.2)
    
    # Score final pondéré
    scoring_df['Score_Final'] = (scoring_df['Score_Performance'] * 0.4 + 
                               scoring_df['Score_Fondamental'] * 0.4 + 
                               scoring_df['Score_Liquidité'] * 0.2)
    
    # Trier par score final
    scoring_df = scoring_df.sort_values('Score_Final', ascending=False)
    
    # Afficher les meilleurs scores
    print("Top 15 Actifs par Score Final:")
    display(scoring_df[['Ticker', 'Nom', 'Pays', 'Secteur', 'Score_Performance', 
                       'Score_Fondamental', 'Score_Liquidité', 'Score_Final']].head(15))
    
    # Sauvegarder les scores
    scores_path = processed_dir / "asset_scores.parquet"
    scoring_df.to_parquet(scores_path)
    print(f"Scores des actifs sauvegardés dans {scores_path}")
    
    # Visualiser les scores
    plt.figure(figsize=(14, 8))
    
    top10 = scoring_df.head(10)
    scores = top10[['Score_Performance', 'Score_Fondamental', 'Score_Liquidité']]
    
    # Créer un graphique à barres empilées
    scores.plot(kind='bar', stacked=True, figsize=(12, 6))
    plt.title('Décomposition des Scores pour les 10 Meilleurs Actifs', fontsize=16)
    plt.xlabel('Ticker')
    plt.ylabel('Score')
    plt.xticks(range(len(top10)), top10['Ticker'], rotation=45)
    plt.grid(True, axis='y')
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

## Conclusion et Prochaines Étapes

Dans ce notebook, nous avons exploré les données pour notre ETF personnalisé ciblant les technologies des marchés émergents. 

### Résumé des Observations
- Nous avons identifié les principales entreprises technologiques des marchés émergents
- Nous avons analysé leurs performances historiques, volatilité et liquidité
- Nous avons examiné leurs fondamentaux et créé un système de scoring pour évaluer les actifs

### Prochaines Étapes
1. **Sélection finale des actifs** - Utiliser le système de scoring et appliquer des filtres supplémentaires selon la stratégie
2. **Pondération du portefeuille** - Implémenter différentes méthodes de pondération (capitalisation, égale, fondamentale)
3. **Backtesting** - Tester la performance historique de l'ETF avec différentes configurations
4. **Analyse des coûts** - Évaluer les coûts de transaction et l'impact sur la performance
5. **Analyse de l'impact sur la liquidité** - Évaluer l'impact potentiel de l'ETF sur la liquidité du marché

Les résultats de cette exploration seront utilisés dans les prochains notebooks pour construire et tester notre ETF personnalisé.